In [1]:
import dspy
from dspy import LM
import random
import litellm
from datetime import datetime, timedelta
import json
import os
import time
import uuid
import ast
from tqdm import tqdm

In [2]:
class GenerateIntentJson(dspy.Signature):
    message = dspy.InputField(desc="Message à interpréter")

    cat = dspy.OutputField(desc="La catégorie : !yi, !py, !cal, !inf")
    ctx = dspy.OutputField(desc="Extrait pertinent ou reformulation")


In [3]:
PROMPT_TEMPLATE = """
Vous êtes un assistant qui catégorise un message utilisateur dans l'une des catégories suivantes :
- !cal : ajout à un calendrier
- !py : appel d'un outil de calcul
- !yi : demande de tirage divinatoire (Yi King, tarot, etc.)
- !inf : question générale ou recherche dans une base de connaissance

Répondez uniquement avec un objet JSON **valide**, contenant :
- "cat" : le code de catégorie (ex: "!cal")
- "ctx" : le message original ou reformulé s'il est ambigu

Message :
{message}

Réponse :
"""


In [4]:
class IntentJsonGenerator(dspy.Module):
    def __init__(self):
        super().__init__()
        self.lm = dspy.settings.lm  # utilise le LM courant

    def format_prompt(self, message):
        return f"""
Vous êtes un assistant intelligent. Classez ce message dans l'une des catégories suivantes :
- !yi : tirage du Yi King ou autre divination
- !py : appel à un programme ou outil Python
- !cal : mise à jour du calendrier
- !inf : recherche d'information
Répondez uniquement par un JSON de la forme :
{{"catégorie": "...", "contexte": "..."}}
Le contexte est généralement le message lui-même.

Message :
{message}
"""

    def forward(self, message):
        prompt = self.format_prompt(message)
        result = self.lm(prompt)[0]
        try:
            import json
            parsed = json.loads(result)
            return parsed
        except Exception:
            return {"catégorie": "?", "contexte": result.strip()}

In [5]:
api_key = os.getenv("OPENAI_API_KEY")
lm = dspy.LM(
    model="gpt-4o",          # modèle reconnu par l’API OpenAI
    api_key=api_key,
    temperature=0.8,
    provider="openai"        # <- obligatoire ici pour désigner le backend
)

dspy.settings.configure(lm=lm)


In [6]:
lm("Dites bonjour")

["Bonjour ! Comment puis-je vous aider aujourd'hui ?"]

In [7]:
class NonStreamingLM(dspy.LM):
    def __call__(self, prompt=None, messages=None, **kwargs):
        kwargs["stream"] = False
        return super().__call__(prompt=prompt, messages=messages, **kwargs)

#philocal = dspy.LM(
#    model="phi3.5:latest",
#    base_url="http://localhost:5000/v1",
#    custom_llm_provider="openai",
#    temperature=0.2
#)
philocal = NonStreamingLM(
    model="phi3.5:latest",
    base_url="http://localhost:5000/v1",
    custom_llm_provider="openai",
    temperature=0.2
)

In [8]:
philocal("Dites bonjour")

["Bonjour! Je suis là pour vous aider avec quoi que ce soit d'autre dont vous avez besoin. Comment puis-je vous assister aujourd'hui?"]

In [9]:
# Charger les données
path = "./intentions-6000-unifiees.jsonl"
actions = []
with open(path, "r", encoding="utf-8") as f:
    json_list = list(f)
    for json_str in json_list:
        result = json.loads(json_str)
        if result['intention'] != '!red' and result['contenu'] != '':
            actions.append(result['contenu'])

print(actions[:12])


["Que dit le Yi King concernant ma décision de me lancer dans l'entrepreneuriat ?", 'Que dit le Yi King concernant ma relation amoureuse actuelle et ses perspectives futures ?', 'Calculez la somme des carrés des nombres de 1 à 50.', 'Quels sont les impacts environnementaux des énergies fossiles ?', 'Quels sont les impacts de la mondialisation sur les économies locales ?', 'Calculez la factorielle de 10.', 'Écrivez un programme pour calculer la factorielle de 10.', 'Planifier une réunion de lancement de projet le 15 janvier à 14h00.', 'Calculez la somme des entiers de 1 à 100.', "Que dit le Yi King concernant ma décision d'accepter une nouvelle opportunité professionnelle ?", 'Calculez la factorielle de 10.', "Planifiez une réunion d'équipe le 15 janvier à 14h00."]


In [12]:
teacher_lm = lm
dspy.settings.configure(lm=teacher_lm)
teacher = IntentJsonGenerator()

json_examples = []
for message in actions[:100]:  # liste de messages bruts
    response = teacher.forward(message)
    json_examples.append(dspy.Example(message=message, output=response).with_inputs("message"))


In [13]:
json_examples[:10]

[Example({'message': "Que dit le Yi King concernant ma décision de me lancer dans l'entrepreneuriat ?", 'output': {'catégorie': '?', 'contexte': '```json\n{"catégorie": "!yi", "contexte": "Que dit le Yi King concernant ma décision de me lancer dans l\'entrepreneuriat ?"}\n```'}}) (input_keys={'message'}),
 Example({'message': 'Que dit le Yi King concernant ma relation amoureuse actuelle et ses perspectives futures ?', 'output': {'catégorie': '?', 'contexte': '```json\n{"catégorie": "!yi", "contexte": "Que dit le Yi King concernant ma relation amoureuse actuelle et ses perspectives futures ?"}\n```'}}) (input_keys={'message'}),
 Example({'message': 'Calculez la somme des carrés des nombres de 1 à 50.', 'output': {'catégorie': '?', 'contexte': '```json\n{"catégorie": "!py", "contexte": "Calculez la somme des carrés des nombres de 1 à 50."}\n```'}}) (input_keys={'message'}),
 Example({'message': 'Quels sont les impacts environnementaux des énergies fossiles ?', 'output': {'catégorie': '?'

In [14]:
def semantic_json_equality(example, pred, trace=None):
    try:
        ref = json.loads(example.output)["intent"]
        gen = json.loads(pred.output)["intent"]
        return ref == gen
    except:
        return False

In [ ]:
student_lm = philocal  # si LoRA, sinon HuggingFace compatible
dspy.settings.configure(lm=student_lm)

student = IntentJsonGenerator()
teleprompter = dspy.teleprompt.BootstrapFewShot(metric=semantic_json_equality)
compiled_student = teleprompter.compile(student, trainset=json_examples)

 47%|████▋     | 47/100 [08:13<10:06, 11.45s/it]

In [ ]:
dspy.settings.configure(lm=philocal)

student = IntentJsonGenerator()
teleprompter = dspy.teleprompt.BootstrapFewShot(metric=semantic_json_equality)
compiled_student = teleprompter.compile(student, trainset=json_examples)

In [ ]:
compiled_student("Planifier une réunion de lancement de projet le 15 janvier à 14h00.")

In [ ]:
compiled_student("Quels sont les impacts de la mondialisation sur les économies locales ?")